# Imports

In [1]:
import numpy as np
import pandas as pd

#!pip install pyEDFlib
import pyedflib
#!pip install ipympl

from scipy.fftpack import fft, ifft, fftfreq
from scipy import signal as sg
from scipy.ndimage.filters import gaussian_filter1d, gaussian_filter
from scipy.stats import binned_statistic, entropy, norm
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import sys
import os
import time
#import pickle
import dill as pickle

import concurrent.futures

from tqdm.notebook import tqdm
from collections import defaultdict
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
#from IPython.display import display
sns.set(context='notebook', style='ticks', palette='bright', font='sans-serif', font_scale=1, color_codes=True, rc=None)
plt.rcParams['figure.figsize'] = (14, 8)

from utility_functions import *
#from lfp_class import LFP
#from pac_class import MyPAC
#from patient_class import Patient

%run lfp_class.py
%run pac_class.py
%run patient_class.py

print("Imported libraries and modules")

In [6]:
with open("path_data.txt") as f:
        data_dir = f.readline()

p5_root_dir = os.path.join(data_dir, "Patient5")

p5 = Patient(name='Patient5', root_dir=p5_root_dir)
files = p5.find_bdf_files()
for filename in files:
    p5.scan_file_annotations(filename, update_file_conditions=True)

List of things to make sure before analysis: 
1) .bdf files are in patient folder (root_dir)
2) annotation files share the same name as .bdf files but with _annotations.txt suffix
3) annotations share the same naming principle: e.g. 1Day OFF RH (Com)
Looking for .bdf files in D:\Alex PAC\Patient5
Found ['Suhih_PD_STN_5day_OFF.bdf', 'Suhih_PD_STN_5day_ON.bdf', 'Suhih_PD_STN_postoper_1day_OFF.bdf', 'Suhih_PD_STN_postoper_1day_ON.bdf']
Reading  Suhih_PD_STN_5day_OFF_annotations.txt
     Onset  Duration                  Annotation   Day L-DOPA  \
0    113.0     300.0  5Day OFF Rest (EyesClosed)  5Day    OFF   
1    422.0     300.0    5Day OFF Rest (EyesOpen)  5Day    OFF   
2    725.0      90.0           5Day OFF RH (Com)  5Day    OFF   
3    826.0      40.0         5Day OFF RH (NoCom)  5Day    OFF   
4    884.0     110.0          5Day OFF RH (Hold)  5Day    OFF   
5   1021.0      30.0          5Day OFF RH (Pass)  5Day    OFF   
6   1089.0      90.0           5Day OFF LH (Com)  5Day    OFF

In [7]:
p5.display_all_annotations()

----1Day----
------OFF------
     Onset  Duration                 State
0      3.0     330.0  [Rest, (EyesClosed)]
1    360.0     300.0    [Rest, (EyesOpen)]
2    685.0      60.0           [RH, (Com)]
3    795.0      45.0         [RH, (NoCom)]
4    861.0     100.0          [RH, (Hold)]
5    977.0      40.0          [RH, (Pass)]
6   1060.0      50.0           [LH, (Com)]
7   1211.0      60.0         [LH, (NoCom)]
8   1296.0      90.0          [LH, (Hold)]
9   1416.0      30.0          [LH, (Pass)]
10  1493.0      70.0                [RHLH]
------ON------
      Onset  Duration                 State
0      7.55     300.0  [Rest, (EyesClosed)]
1    341.50     270.0    [Rest, (EyesOpen)]
2    650.00      70.0           [RH, (Com)]
3    760.00      50.0         [RH, (NoCom)]
4    846.00     100.0          [RH, (Hold)]
5    968.00      35.0          [RH, (Pass)]
6   1026.00      90.0           [LH, (Com)]
7   1140.00      60.0         [LH, (NoCom)]
8   1216.00     100.0          [LH, (Hold)]


In [8]:
p5.get_preprocessed_lfps(verbose=False)

Started reading D:\Alex PAC\Patient5\Suhih_PD_STN_5day_ON.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.75it/s]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  2000.0
Reading done, 4.7 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


  0%|          | 0/30 [00:00<?, ?it/s]

Started reading D:\Alex PAC\Patient5\Suhih_PD_STN_5day_OFF.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


...

  0%|          | 0/30 [00:00<?, ?it/s]

Started reading D:\Alex PAC\Patient5\Suhih_PD_STN_postoper_1day_ON.bdf
Channels: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


Reading Channels: 100%|████████████████████████████████████████████████████████████████| 16/16 [00:13<00:00,  1.16it/s]


EEG A2-R_1-9[A2]
EEG F8-R_2C-15[F
EEG T4-R_2B-12[T
EEG T6-R_2A-10[T
EEG F4-R_3C-16[F
EEG C4-R_3B-14[C
EEG P4-R_3A-11[P
EEG O2-R_4-13[O2
EEG A1-R_1-1[A1]
EEG F7-R_2C-7[F7
EEG T3-R_2B-4[T3
EEG T5-R_2A-2[T5
EEG F3-R_3C-8[F3
EEG C3-R_3B-6[C3
EEG P3-R_3A-3[P3
EEG O1-R_4-5[O1]
Sampling frequency:  2000.0
Reading done, 15.2 sec
Started creating bipolar signals

R1-2C created
R1-2B created
R1-2A created
R2C-2B created
R2B-2A created
R2A-2C created
R2C-3C created
R2B-3B created
R2A-3A created
R3C-3B created
R3B-3A created
R3A-3C created
R4-3C created
R4-3B created
R4-3A created
L1-2C created
L1-2B created
L1-2A created
L2C-2B created
L2B-2A created
L2A-2C created
L2C-3C created
L2B-3B created
L2A-3A created
L3C-3B created
L3B-3A created
L3A-3C created
L4-3C created
L4-3B created
L4-3A created


  0%|          | 0/30 [00:00<?, ?it/s]

In [11]:
p5.conditions

{'1Day OFF LH (Com)',
 '1Day OFF LH (Hold)',
 '1Day OFF LH (NoCom)',
 '1Day OFF LH (Pass)',
 '1Day OFF RH (Com)',
 '1Day OFF RH (Hold)',
 '1Day OFF RH (NoCom)',
 '1Day OFF RH (Pass)',
 '1Day OFF RHLH',
 '1Day OFF Rest (EyesClosed)',
 '1Day OFF Rest (EyesOpen)',
 '1Day ON LH (Com)',
 '1Day ON LH (Hold)',
 '1Day ON LH (NoCom)',
 '1Day ON LH (Pass)',
 '1Day ON RH (Com)',
 '1Day ON RH (Hold)',
 '1Day ON RH (NoCom)',
 '1Day ON RH (Pass)',
 '1Day ON RHLH',
 '1Day ON Rest (EyesClosed)',
 '1Day ON Rest (EyesOpen)',
 '5Day OFF LH (Com)',
 '5Day OFF LH (Hold)',
 '5Day OFF LH (NoCom)',
 '5Day OFF LH (Pass)',
 '5Day OFF RH (Com)',
 '5Day OFF RH (Hold)',
 '5Day OFF RH (NoCom)',
 '5Day OFF RH (Pass)',
 '5Day OFF RHLH',
 '5Day OFF Rest (EyesClosed)',
 '5Day OFF Rest (EyesOpen)',
 '5Day ON LH (Com)',
 '5Day ON LH (Hold)',
 '5Day ON LH (NoCom)',
 '5Day ON LH (Pass)',
 '5Day ON RH (Com)',
 '5Day ON RH (Hold)',
 '5Day ON RH (NoCom)',
 '5Day ON RH (Pass)',
 '5Day ON RHLH 1',
 '5Day ON RHLH 2',
 '5Day ON R

In [12]:
new_condition_name = "1Day OFF Rest 180sec"
conditions_to_merge = ["1Day OFF Rest (EyesOpen)", "1Day OFF Rest (EyesClosed)"]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "1Day ON Rest 180sec"
conditions_to_merge = ["1Day ON Rest (EyesOpen)", "1Day ON Rest (EyesClosed)"]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day OFF RH Move 180sec"
conditions_to_merge = ["1Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day OFF LH Move 180sec"
conditions_to_merge = ["1Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "1Day ON RH Move 180sec"
conditions_to_merge = ["1Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "1Day ON LH Move 180sec"
conditions_to_merge = ["1Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


## 5 DAY

new_condition_name = "5Day OFF Rest 180sec"
conditions_to_merge = ["5Day OFF Rest (EyesOpen)", "5Day OFF Rest (EyesClosed)"]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON Rest 180sec"
conditions_to_merge = ["5Day ON Rest (EyesOpen)", "5Day ON Rest (EyesClosed)"]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day OFF RH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day OFF LH Move 180sec"
conditions_to_merge = ["5Day OFF " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)


new_condition_name = "5Day ON RH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["RH (Com)", "RH (NoCom)", "RH (Hold)", "RH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

new_condition_name = "5Day ON LH Move 180sec"
conditions_to_merge = ["5Day ON " + s for s in ["LH (Com)", "LH (NoCom)", "LH (Hold)", "LH (Pass)"]]
p5.merge_conditions(conditions_to_merge, new_condition_name, total_duration=180)

Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L2C-3C
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L2B-2A
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L3A-3C
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R2A-2C
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R2A-3A
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R3A-3C
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: L2A-3A
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R1-2A
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R4-3B
Updating condition
Adding LFP to Patient5 object. 
Condition: 1Day OFF Rest 180sec 
Placement: R4-3A
Up

In [13]:
for condition in p5.conditions:
    if "180sec" in condition:
        print(condition)

5Day OFF RH Move 180sec
1Day ON LH Move 180sec
1Day OFF LH Move 180sec
1Day ON RH Move 180sec
5Day ON RH Move 180sec
5Day ON LH Move 180sec
1Day ON Rest 180sec
1Day OFF RH Move 180sec
1Day OFF Rest 180sec
5Day OFF Rest 180sec
5Day ON Rest 180sec
5Day OFF LH Move 180sec


In [14]:
p5.save()

Saving Patient5 object to D:\Alex PAC\Patient5\Patient5.pkl ...
Pickling Patient5 without ['pac']
Done, 2.263138771057129 sec
File size: 3022.9362802505493 MB
Returning filepath for saved file


'D:\\Alex PAC\\Patient5\\Patient5.pkl'